<a href="https://colab.research.google.com/github/koyomin9zx/CS331.K11-Flowers-Recognition/blob/master/testModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#kết nối driver
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ln -s /content/drive/'My Drive'/ML/backup /content

In [0]:
!unzip /content/drive/'My Drive'/ML/dataset.zip

In [0]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.externals import joblib
import cv2
from keras import models
from keras import layers
from keras import optimizers

In [6]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))

# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)

# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
model = VGG16(weights='imagenet', include_top=False)

# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)


# Dùng pre-trained model để lấy ra các feature của ảnh
features = model.predict(list_image)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features = features.reshape((features.shape[0], 512*7*7))





58892288/58889256 [==============================] - 5s 0us/step








In [0]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [0]:
svm_gauss = joblib.load('backup/svm_gauss.pkl')
svm_linear=joblib.load('backup/svm_linear.pkl')
svm_poly=joblib.load('backup/svm_poly.pkl')

In [11]:

#kiểm tra kết quả của linearSVM
y_pred = svm_linear.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("\n")
print(classification_report(y_test,y_pred))

[[146   4   1   1   1]
 [  1 210   0   0   1]
 [  1   0 144   1   2]
 [  2   0   2 148   0]
 [  0   1   2   0 197]]


              precision    recall  f1-score   support

           0       0.97      0.95      0.96       153
           1       0.98      0.99      0.98       212
           2       0.97      0.97      0.97       148
           3       0.99      0.97      0.98       152
           4       0.98      0.98      0.98       200

    accuracy                           0.98       865
   macro avg       0.98      0.98      0.98       865
weighted avg       0.98      0.98      0.98       865



In [52]:
#kiểm tra kết quả của SVM với kernel polynominal
y_pred_poly = svm_poly.predict(X_test)
print(confusion_matrix(y_test,y_pred_poly))
print("\n")
print(classification_report(y_test,y_pred_poly))

[[136  17   0   0   0]
 [  1 211   0   0   0]
 [  0  24 123   0   1]
 [  1  23   0 128   0]
 [  1  32   2   0 165]]


              precision    recall  f1-score   support

           0       0.98      0.89      0.93       153
           1       0.69      1.00      0.81       212
           2       0.98      0.83      0.90       148
           3       1.00      0.84      0.91       152
           4       0.99      0.82      0.90       200

    accuracy                           0.88       865
   macro avg       0.93      0.88      0.89       865
weighted avg       0.92      0.88      0.89       865



In [13]:
#kiểm tra kết quả của SVM với kernel gaussian
y_pred_gauss = svm_gauss.predict(X_test)
print(confusion_matrix(y_test,y_pred_gauss))
print("\n")
print(classification_report(y_test,y_pred_gauss))

[[119  34   0   0   0]
 [  0 212   0   0   0]
 [  0  28 120   0   0]
 [  0  27   0 125   0]
 [  0  37   1   0 162]]


              precision    recall  f1-score   support

           0       1.00      0.78      0.88       153
           1       0.63      1.00      0.77       212
           2       0.99      0.81      0.89       148
           3       1.00      0.82      0.90       152
           4       1.00      0.81      0.90       200

    accuracy                           0.85       865
   macro avg       0.92      0.84      0.87       865
weighted avg       0.91      0.85      0.86       865



In [30]:
#Kiểm tra kết quả của softmax classifer 
y_pred_gauss=softmax.predict(X_test)
rounded_pred=np.argmax(y_pred_gauss,axis=1)
print(confusion_matrix(y_test,rounded_pred))
print("\n")
print(classification_report(y_test,rounded_pred))

[[136  12   2   2   1]
 [  4 202   1   4   1]
 [  1   2 139   0   6]
 [  7   7   2 133   3]
 [  3   7  15   5 170]]


              precision    recall  f1-score   support

           0       0.90      0.89      0.89       153
           1       0.88      0.95      0.91       212
           2       0.87      0.94      0.91       148
           3       0.92      0.88      0.90       152
           4       0.94      0.85      0.89       200

    accuracy                           0.90       865
   macro avg       0.90      0.90      0.90       865
weighted avg       0.90      0.90      0.90       865



In [57]:
#Test ảnh
def testImage(path):
  model1 = VGG16(weights='imagenet', include_top=False)
  img = cv2.imread(path)
  img = cv2.resize(img,(224,224))
  img = img_to_array(img)
  img = np.expand_dims(img, 0)
  img = imagenet_utils.preprocess_input(img)
  imgfeat = model1.predict(img)
  imgfeat = imgfeat.reshape((imgfeat.shape[0], 512*7*7))
  return imgfeat

def nameFlower(pred):
  if(pred[0]==0):
    return "Hoa Cúc"
  if(pred[0]==1):
    return "Hoa Bồ Công Anh"
  if(pred[0]==2):
    return "Hoa Hồng"
  if(pred[0]==3):
    return "Hoa Hướng Dương"
  if(pred[0]==4):
    return "Hoa Tuy Líp"



imgfeat=testImage('rose.jpg')


preds1 = softmax.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán sofmax: ',nameFlower(np.argmax(preds1,axis=1)))

preds2 = svm_linear.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán linearSVM: ',nameFlower(preds2))

preds3 = svm_gauss.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán gaussian: ',nameFlower(preds3))

preds4 = svm_poly.predict(imgfeat.reshape(1,-1))
print('Giá trị dự đoán poly: ',nameFlower(preds4))

Giá trị dự đoán sofmax:  Hoa Hồng
Giá trị dự đoán linearSVM:  Hoa Hồng
Giá trị dự đoán gaussian:  Hoa Bồ Công Anh
Giá trị dự đoán poly:  Hoa Bồ Công Anh
